In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
pip install -q keras

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [ ]:
import keras
import mtcnn
# print version
print(mtcnn.__version__)
print(keras.__version__)

0.1.0
2.4.3


In [ ]:
#Load all necessary libraries
import numpy as np
import cv2 # opencv
import os 
from keras.models import Sequential,Model
from keras.layers import Convolution2D,Activation,Dense,MaxPooling2D,ZeroPadding2D,Flatten,BatchNormalization,Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image

Looking for Images in dataset
(for debugging purpose only)

In [ ]:
for imgfolder in os.listdir('/content/drive/My Drive/bollywood_celeb_faces_0'): #iterate thru each of the 5 celeb folders
    for filename in os.listdir('/content/drive/My Drive/bollywood_celeb_faces_0' + '/'+ imgfolder):# iterate thru each image in a celeb folder
        filename = '/content/drive/My Drive/bollywood_celeb_faces_0' + imgfolder + '/' + filename # build the path to the image file
        print(filename) # print the filename read. For debugging purpose only

/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/107.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/104.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/103.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/108.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/116.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/110.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/105.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/118.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/111.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/75.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/80.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/89.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/133.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam_Kapoor/94.jpg
/content/drive/My Drive/bollywood_celeb_faces_0Sonam

In [ ]:
#Fface extraction using MTCNN
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    for i in range(0,len(results)):
      x1, y1, width, height = results[0]['box']
    # deal with negative pixel index
      x1, y1 = abs(x1), abs(y1)
      x2, y2 = x1 + width, y1 + height
       # extract the face
      face = pixels[y1:y2, x1:x2]
       # resize pixels to the model size
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = np.asarray(image)
      return face_array
      
   
   

In [ ]:
#Loading dataset
def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path= dir + '/' + filename
        face = extract_face(path)
        faces.append(face)
    return faces

       
def load_dataset(dir):
    # list for faces and labels
    X = list()
    y=list()
    for subdir in os.listdir(dir):
        path = (dir + '/'+ subdir)
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces),subdir) ) # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

# load train dataset
trainX, trainy = load_dataset('/content/drive/My Drive/bollywood_celeb_faces_0')
print(trainX.shape, trainy.shape)

loaded 34 sample for class: Sonam_Kapoor
loaded 16 sample for class: Varun_Dhawan
loaded 12 sample for class: Taapsee_Pannu
loaded 27 sample for class: Sushant_Singh_Rajput
loaded 8 sample for class: Shilpa_Shetty
loaded 13 sample for class: Shah_Rukh_Khan
loaded 30 sample for class: Salman_Khan
loaded 22 sample for class: Rajkummar_Rao
loaded 11 sample for class: Preity_Zinta
loaded 19 sample for class: Ranveer_Singh
loaded 30 sample for class: Ranbir_Kapoor
loaded 8 sample for class: Rani_Mukerji
loaded 14 sample for class: Priyanka_Chopra
loaded 36 sample for class: Kartik_Aaryan
loaded 32 sample for class: Manoj_Bajpayee
loaded 51 sample for class: Kareena_Kapoor
loaded 8 sample for class: Katrina_Kaif
loaded 34 sample for class: Madhuri_Dixit
loaded 36 sample for class: Kiara_Advani


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 206 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 724 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 170 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 178 bytes but o

loaded 33 sample for class: Karisma_Kapoor
loaded 12 sample for class: Irrfan_Khan
loaded 12 sample for class: Hrithik_Roshan
loaded 16 sample for class: Disha_Patani
loaded 16 sample for class: Deepika_Padukone
loaded 12 sample for class: Aishwarya_Rai
loaded 20 sample for class: Anushka_Sharma
loaded 16 sample for class: Akshay_Kumar
loaded 23 sample for class: Alia_Bhatt
loaded 18 sample for class: Anil_Kapoor


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 15191503488 bytes but only got 0. Skipping tag 8126
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 10. 
  warnings.warn(str(msg))


loaded 25 sample for class: Amitabh_Bachchan
loaded 37 sample for class: Ayushmann_Khurrana
loaded 52 sample for class: Aamir_Khan
(733, 224, 224, 3) (733,)


In [ ]:

# save and compress the dataset for further use
np.savez_compressed('celebrity-faces-dataset1.npz', trainX, trainy)

In [ ]:
# load the face dataset
data = np.load('/content/celebrity-faces-dataset1.npz')
trainX, trainy= data['arr_0'], data['arr_1']
print('Loaded: ', trainX.shape, trainy.shape)

Loaded:  (733, 224, 224, 3) (733,)
